## Abfragen in der ClassicModels-Datenbank

### Entity Relationship Diagram:

![Entity Relationship Diagram](./img/ERD_classic_models.png)

### Einfache Abfragen

1. Erstellung einer Abfrage, die die Namen aller Mitarbeiter und ihre jeweiligen Titel ausgibt.

In [ ]:
SELECT firstname, lastname, jobtitle 
FROM employees;

2. Abruf aller Informationen zu Produkten aus der Produktlinie „Motorcycles“.

In [ ]:
SELECT * 
FROM products
WHERE productline = 'Motorcycles';

3. Ermittlung der Gesamtanzahl der Motorräder, die aktuell auf Lager sind.

In [ ]:
SELECT
	sum(quantityinstock) as Anzahl_Motorraeder_auf_Lager
FROM products
WHERE productline = 'Motorcycles';

4. Berechnung des günstigsten, teuersten und durchschnittlichen Preises der Motorräder.

In [ ]:
SELECT
	min(buyprice) as günstigstes_Motorrad,
	max(buyprice) as teuerstes_Motorrad,
	avg(buyprice) as durchschnittlicher_Preis_Motorrad
FROM products
WHERE productline = 'Motorcycles';

5. Identifikation von Bestellungen, die später versandt wurden, als ursprünglich gefordert.

In [ ]:
SELECT * 
FROM orders
WHERE requireddate < shippeddate;

6. Ermittlung der drei häufigsten Kommentare zu Bestellungen.

In [ ]:
SELECT comments, count(comments) as anzahl  
FROM orders
GROUP BY comments
ORDER BY anzahl desc
LIMIT 3;

7. Berechnung der durchschnittlichen Lieferzeit zwischen Auftragserteilung und Auslieferung.

In [ ]:
SELECT 
	avg(shippeddate - orderdate) as durchschnittliche_Lieferungzeit
FROM orders;

### Komplexere Abfragen

8. Ermittlung des Sales-Mitarbeiters mit dem höchsten generierten Zahlungsbetrag.

In [ ]:
SELECT e.employeenumber, e.firstname, e.lastname, sum(pm.amount) as Zahlungsbetrag
FROM payments pm
INNER JOIN customers c USING(customernumber)
INNER JOIN employees e ON c.salesrepemployeenumber = e.employeenumber
GROUP BY e.employeenumber, e.firstname, e.lastname
ORDER BY Zahlungsbetrag DESC
LIMIT 1;

9. Identifikation von Produkten, die bisher von keinem Kunden bestellt wurden.

In [ ]:
SELECT p.productcode, od.quantityordered
FROM products p
LEFT JOIN orderdetails od USING(productcode)
WHERE od.quantityordered IS NULL;

10. Analyse der Verkaufsmenge und Gewinnspanne aller unterschiedlichen Porsche-Produkte.

In [ ]:
SELECT 
	p.productname, 
	sum(od.quantityordered) as "Verkaufsmenge",
	sum((p.MSRP - p.buyprice) * od.quantityordered) as "Erw. Gewinn",
	sum((od.priceeach - p.buyprice) * od.quantityordered) as "Erz. Gewinn"
FROM products p
INNER JOIN orderdetails od USING(productcode)
WHERE productname ILIKE '%porsche%'
GROUP BY productname
ORDER BY "Verkaufsmenge" asc;

### Abfragen mit Subqueries und CTEs

11. Berechnung des Gesamtumsatzes pro Kunde und des prozentualen Anteils am Gesamtumsatz.

In [ ]:
WITH Gesamtumsatz_pro_Kunde as (
					SELECT 
						c.customernumber as Kunde,
						sum(od.quantityordered * od.priceeach) as Ausgaben_pro_Kunde
					FROM customers c
					INNER JOIN orders o         USING(customernumber)
					INNER JOIN orderdetails od  USING(ordernumber)
					GROUP BY Kunde
	),
	Gesamtumsatz as (
				SELECT 
					sum(Ausgaben_pro_Kunde) as Gesamtumsatz
				FROM Gesamtumsatz_pro_Kunde
	)
SELECT
	Kunde,
	Ausgaben_pro_Kunde,
	Gesamtumsatz,
	(Ausgaben_pro_Kunde/Gesamtumsatz)*100 as Anteil_am_Gesamtumsatz
FROM Gesamtumsatz_pro_Kunde
INNER JOIN Gesamtumsatz ON TRUE
;

12. Berechnung des durchschnittlichen Zahlungsbetrags pro Verkaufsperson und Standort.

In [ ]:
SELECT 
	city,
	country,
	SalesRep,
	Zahlungsbetrag,
 	(Zahlungsbetrag / SalesRep) as Durchschnittlicher_Umsatz_Pro_Standort
FROM
	(SELECT 
			o.city,
			o.country,
			count(DISTINCT e.employeenumber) as SalesRep,
			sum(pm.amount) as Zahlungsbetrag
		FROM employees e
		INNER JOIN offices o USING(officecode)
		INNER JOIN customers c ON c.salesrepemployeenumber = e.employeenumber
		INNER JOIN payments pm USING(customernumber)
		WHERE e.jobtitle = 'Sales Rep'
		GROUP BY o.country, o.city);
		
-- Alternative ohne Subquery
SELECT 
	o.city,
	o.country,
	count(DISTINCT e.employeenumber) as SalesRep,
	sum(pm.amount) as Zahlungsbetrag,
	(sum(pm.amount) / count(DISTINCT e.employeenumber)) AS Durchschnittlicher_Umsatz_Pro_Standort
FROM employees e
INNER JOIN offices o USING(officecode)
INNER JOIN customers c ON c.salesrepemployeenumber = e.employeenumber
INNER JOIN payments pm USING(customernumber)
WHERE e.jobtitle = 'Sales Rep'
GROUP BY o.country, o.city;

13. Vergleich des monatlichen Umsatzes von 2003 und 2004 sowie Berechnung der Differenz.

In [ ]:
WITH 
	Umsatz_von_2003 AS (
		SELECT
			EXTRACT(MONTH FROM o.orderdate) AS Monat,
			sum(od.quantityordered * od.priceeach) as Umsatz_2003
		FROM orderdetails od
		INNER JOIN orders o USING(ordernumber)
		WHERE EXTRACT(YEAR FROM o.orderdate) = 2003 
		GROUP BY Monat
		ORDER BY Monat asc),
	Umsatz_von_2004 AS (
		SELECT
			EXTRACT(MONTH FROM o.orderdate) AS Monat,
			sum(od.quantityordered * od.priceeach) as Umsatz_2004
		FROM orderdetails od
		INNER JOIN orders o USING(ordernumber)
		WHERE EXTRACT(YEAR FROM o.orderdate) = 2004 
		GROUP BY Monat
		ORDER BY Monat asc)
SELECT
	 Monat, Umsatz_2003, Umsatz_2004,
	 (Umsatz_2004 - Umsatz_2003) as Differenz
FROM Umsatz_von_2003
INNER JOIN Umsatz_von_2004 USING(Monat)
GROUP BY Monat, Umsatz_2003, Umsatz_2004;


-- Alternative mit Subquery
SELECT *,
	s2004, umsatz_2004 - s2003.umsatz_2003 as Differenz
FROM (SELECT
			EXTRACT(MONTH FROM o.orderdate) AS Monat,
			sum(od.quantityordered * od.priceeach) as Umsatz_2003
		FROM orderdetails od
		INNER JOIN orders o USING(ordernumber)
		WHERE EXTRACT(YEAR FROM o.orderdate) = 2003 
		GROUP BY Monat
		ORDER BY Monat asc) as s2003
INNER JOIN (SELECT
			EXTRACT(MONTH FROM o.orderdate) AS Monat,
			sum(od.quantityordered * od.priceeach) as Umsatz_2004
		FROM orderdetails od
		INNER JOIN orders o USING(ordernumber)
		WHERE EXTRACT(YEAR FROM o.orderdate) = 2004 
		GROUP BY Monat
		ORDER BY Monat asc) as s2004 USING(Monat)
;

-- Alternative mit CASE WHEN
SELECT
	EXTRACT(MONTH FROM orderdate) as Monat,
	SUM(CASE WHEN EXTRACT(YEAR FROM o.orderdate) = 2003 
	THEN quantityordered * priceeach ELSE 0 END) as Umsatz2003,
	SUM(CASE WHEN EXTRACT(YEAR FROM o.orderdate) = 2004 
	THEN quantityordered * priceeach ELSE 0 END) as Umsatz2004
FROM orders o
JOIN orderdetails od USING(ordernumber)
GROUP BY EXTRACT(MONTH FROM orderdate)
ORDER BY Monat;

14. Identifikation von Produktpaaren, die häufig gemeinsam gekauft werden.

In [ ]:
SELECT 
	od.productcode, 
	od2.productcode,
	count(*) as Zusammengekauft 
FROM orderdetails od
INNER JOIN orderdetails od2 ON od.productcode < od2.productcode AND od.ordernumber = od2.ordernumber
GROUP BY od.productcode, od2.productcode
ORDER BY Zusammengekauft desc;

--Alternative mit Subquery
SELECT 
	paare.item1,
	paare.item2,
	count(item2) as Anzahl
FROM
(
SELECT item1.productcode as item1, item2.productcode as item2
FROM orderdetails item1
INNER JOIN orderdetails item2
	ON item1.productcode < item2.productcode
	AND item1.ordernumber = item2.ordernumber
) as paare
GROUP BY paare.item1, paare.item2
ORDER BY Anzahl DESC;

15. Bestimmung von Produkten, die im Dezember 2003, aber nicht im Dezember 2004 verkauft wurden.

In [ ]:
SELECT
	verkauf_2003.productcode,
    verkauf_2003.Anzahl_2003,
	verkauf_2004.Anzahl_2004
FROM   (SELECT 
			od.productcode as productcode,
			count(od.productcode) as Anzahl_2003
		FROM orderdetails od 
		INNER JOIN orders o    USING(ordernumber)
		WHERE EXTRACT(YEAR FROM o.orderdate) = 2003 AND EXTRACT(MONTH FROM o.orderdate) = 12
		GROUP BY od.productcode) AS verkauf_2003
LEFT JOIN (SELECT 
			od.productcode as productcode,
			count(od.productcode) as Anzahl_2004
		FROM orderdetails od 
		INNER JOIN orders o    USING(ordernumber)
		WHERE EXTRACT(YEAR FROM o.orderdate) = 2004 AND EXTRACT(MONTH FROM o.orderdate) = 12
		GROUP BY od.productcode) AS verkauf_2004 
		USING(productcode)
WHERE Anzahl_2004 IS NULL;

-- Alternative 
SELECT * 
FROM (
	SELECT DISTINCT od.productcode
	FROM orders o 
	INNER JOIN orderdetails od USING(ordernumber)
	WHERE orderdate BETWEEN '2003-12-01' AND '2003-12-31'
	) res2003
LEFT JOIN (SELECT DISTINCT od.productcode
	FROM orders o 
	INNER JOIN orderdetails od USING(ordernumber)
	WHERE orderdate BETWEEN '2004-12-01' AND '2004-12-31' 
	) res2004
	ON res2003.productcode = res2004.productcode
WHERE res2004 IS NULL;

16. Berechnung des durchschnittlichen Bestellwerts aller Bestellungen aus dem Jahr 2004.

In [ ]:
WITH Bestellungen_2004 AS (
			SELECT
				ordernumber,
				sum(od.quantityordered * od.priceeach) as Bestellungen
			FROM orderdetails od
			INNER JOIN orders o USING(ordernumber)
			WHERE EXTRACT(YEAR FROM o.orderdate) = 2004
			GROUP BY ordernumber
			)
SELECT 
	avg(Bestellungen) as Durchschnitt
FROM Bestellungen_2004;

17. Ermittlung der Bestellungen aus 2004, deren Bestellwert über dem durchschnittlichen Bestellwert liegt.

In [ ]:
WITH Bestellungen_2004 AS (
					SELECT
						ordernumber,
						sum(od.quantityordered * od.priceeach) as Bestellungen
					FROM orderdetails od
					INNER JOIN orders o USING(ordernumber)
					WHERE EXTRACT(YEAR FROM o.orderdate) = 2004
					GROUP BY ordernumber),
	Bestellwerte AS (
					SELECT 
						avg(Bestellungen) as Durchschnitt
					FROM Bestellungen_2004) 
SELECT 
	ordernumber,
	Bestellungen
FROM Bestellungen_2004
INNER JOIN Bestellwerte ON TRUE
WHERE Bestellungen > Durchschnitt
GROUP BY ordernumber, Bestellungen
ORDER BY Bestellungen desc;